# Vector::iterator - begin/end correctness

## The Problem with nullptr and Pointer Arithmetic

**Issue**: In a default-constructed `Vector`, `a_` is `nullptr` and `size_` is `0`. Computing `end()` as `a_ + size_` performs pointer arithmetic on a null pointer, which is undefined behavior—even when `size_ == 0`.

**Why it's UB**: The C++ standard doesn't allow pointer arithmetic on null pointers. Even `nullptr + 0` is undefined behavior.

**Additional issue**: The default constructor sets `capacity_ = 10` but doesn't allocate storage, so the first `push_back` will try to write to unallocated memory.

---

## Two Clean Solutions

### Option 1: Always Maintain a Valid Buffer (Eager Allocation)
- Allocate storage in the default constructor with the initial `capacity_` (e.g., 10 elements).
- Then `begin() = a_` and `end() = a_ + size_` are always safe.
- **Pros**: Simplest mental model; mirrors `std::vector` implementations that keep non-null `data()` even when empty.
- **Cons**: Uses a small amount of memory upfront (fine for practice).

### Option 2: Lazy Allocation with Guards
- Keep `a_` as `nullptr` until first allocation, but:
  - Set `capacity_ = 0` initially (not 10).
  - In `end()`: if `a_` is `nullptr`, return `iterator(a_)` (same as `begin()`) without doing arithmetic.
  - In `push_back()`: handle `capacity_ == 0` case by growing to at least 1 before writing.
- **Pros**: No memory allocated until needed.
- **Cons**: Must be careful with growth from capacity 0 and avoid any pointer arithmetic on nullptr.

---

## Invariants to Maintain

**Invariant A (eager)**: `a_` is never null after construction. Then `begin()/end()` can freely use `a_` and `a_ + size_`.

**Invariant B (lazy)**: `a_` can be null, but:
- `begin()` returns `iterator(a_)`
- `end()` returns `iterator(a_)` when `a_` is null (no arithmetic), otherwise `iterator(a_ + size_)`
- `push_back()` must allocate when `capacity_ == 0` before writing
- `capacity_` must accurately reflect real storage (0 when null, not 10)

---

## Sanity Checklist
- ✓ Default ctor + `push_back(42)` works without crashing
- ✓ `begin() == end()` on an empty vector, without any pointer arithmetic on nullptr
- ✓ After first allocation, `end()` is exactly one past the last element (`a_ + size_`)
- ✓ Optional: mark `begin()/end()` as `noexcept` once UB path is eliminated

---

## Where to Use noexcept in Vector::iterator

Mark functions `noexcept` when, under documented preconditions, they cannot throw exceptions.

### Safe for noexcept (operations that don't throw):
- `iterator(int*) noexcept` — just stores a pointer
- `int& operator*() const noexcept` — dereferencing doesn't throw (UB if invalid, but not an exception)
- `iterator& operator++() noexcept` — pointer increment is non-throwing
- `iterator operator++(int) noexcept` — makes trivial copy and increments
- `bool operator==(const iterator&) const noexcept` — pointer comparisons don't throw
- `bool operator!=(const iterator&) const noexcept` — pointer comparisons don't throw
- `int* GetPtr() const noexcept` — just returns the pointer
- `iterator begin() noexcept` — constructs iterator from pointer (once UB is fixed)
- `iterator end() noexcept` — constructs iterator from pointer (once UB is fixed)

### Key Points:
- **Built-in ops don't throw**: pointer arithmetic, comparisons, trivial copies are all safe
- **UB is not an exception**: dereferencing invalid pointers is UB, not an exception—noexcept only means "won't throw"
- **Document preconditions**: e.g., "iterator must be dereferenceable" for `operator*`
- **Watch for**: allocations, bounds checking, user code that might throw